<a href="https://colab.research.google.com/github/egnsuresh/Spark_Practice/blob/master/array_list_to_string_type_without_duplicates_(convert_data_and_it's_type)_functions_in_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



---



---



# 1. PySpark installation , spark session object creation and importing common functions.

---

---





In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=8367083811b6ff31bafaf49ac64eb628cb2e409e66c8f0e74610391966c49bce
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
import pyspark
spark=pyspark.sql.SparkSession.builder.getOrCreate()

In [5]:
from pyspark.sql import functions as f

# 2. Sample data creation & Data profiling

---


---




In [16]:
# prompt: create sample 10 different records of dataframe with cust_name,cif,tran_type, branch_name,outlet_name,terminal_name,tran_amount and timestamp

data = [('John Doe', '1234567890', 'Deposit', 'New York', 'ABC Bank outlet1', '123', 100.0, '2022-01-01'),
 ('Jane Doe', '0987654321', 'Withdrawal', 'Los Angeles', 'XYZ Bank outlet1', '456', 50.0, '2022-01-02'),
 ('Michael Jones', '2345678901', 'Transfer', 'Chicago', 'PQR Bank outlet1', '789', 25.0, '2022-01-03'),
('Maccha Maama', '1234567890', 'Deposit', 'New York', 'ABC Bank outlet1', '123', 200.0, '2022-01-01'),
 ('Sarah Smith', '3456789012', 'Bill payment', 'Boston', 'STU Bank outlet1', '012', 15.0, '2022-01-04'),
 ('William Johnson', '4567890123', 'ATM withdrawal', 'Philadelphia', 'UVW Bank outlet1', '345', 10.0, '2022-01-05'),
 ('Mary Brown', '5678901234', 'Online purchase', 'San Francisco', 'XYZ Bank outlet1', '678', 5.0, '2022-01-06'),
 ('David Miller', '6789012345', 'ACH transfer', 'Dallas', 'ABC Bank outlet1', '901', 2.5, '2022-01-07'),
 ('Robert Anderson', '7890123456', 'Check deposit', 'Houston', 'PQR Bank outlet1', '234', 1.0, '2022-01-08'),
 ('Jessica Garcia', '8901234567', 'Money order', 'Atlanta', 'STU Bank outlet1', '567', 0.5, '2022-01-09'),
 ('Matthew Rodriguez', '9012345678', 'Payroll', 'Seattle', 'UVW Bank outlet1', '890', 0.25, '2022-01-10')]

cust_df = spark.createDataFrame(data, ['cust_name', 'cif', 'tran_type', 'branch_name'\
, 'outlet_name', 'terminal_name', 'tran_amount', 'tran_date'])
cust_df.show(20,False)
cust_df.printSchema()
cust_df.count()
cust_df.select("tran_type","outlet_name").show(11,False)

+-----------------+----------+---------------+-------------+----------------+-------------+-----------+----------+
|cust_name        |cif       |tran_type      |branch_name  |outlet_name     |terminal_name|tran_amount|tran_date |
+-----------------+----------+---------------+-------------+----------------+-------------+-----------+----------+
|John Doe         |1234567890|Deposit        |New York     |ABC Bank outlet1|123          |100.0      |2022-01-01|
|Jane Doe         |0987654321|Withdrawal     |Los Angeles  |XYZ Bank outlet1|456          |50.0       |2022-01-02|
|Michael Jones    |2345678901|Transfer       |Chicago      |PQR Bank outlet1|789          |25.0       |2022-01-03|
|Maccha Maama     |1234567890|Deposit        |New York     |ABC Bank outlet1|123          |200.0      |2022-01-01|
|Sarah Smith      |3456789012|Bill payment   |Boston       |STU Bank outlet1|012          |15.0       |2022-01-04|
|William Johnson  |4567890123|ATM withdrawal |Philadelphia |UVW Bank outlet1|345

In [17]:
#importing required window class
from pyspark.sql.window import Window

outlet_df=cust_df.withColumn("outlet_tran_list"\
                             ,f.collect_list("tran_type")\
                             .over(Window.partitionBy("outlet_name")))\
                             .select("outlet_name","outlet_tran_list")\
                            .dropDuplicates()

outlet_df.show(10,False)
outlet_df.printSchema()

+----------------+--------------------------------+
|outlet_name     |outlet_tran_list                |
+----------------+--------------------------------+
|ABC Bank outlet1|[Deposit, Deposit, ACH transfer]|
|PQR Bank outlet1|[Transfer, Check deposit]       |
|STU Bank outlet1|[Bill payment, Money order]     |
|UVW Bank outlet1|[ATM withdrawal, Payroll]       |
|XYZ Bank outlet1|[Withdrawal, Online purchase]   |
+----------------+--------------------------------+

root
 |-- outlet_name: string (nullable = true)
 |-- outlet_tran_list: array (nullable = false)
 |    |-- element: string (containsNull = false)



# 3. use cases of  array_distinct and concat_ws

---



---



<table>
<tr>
<td>
<pre>
+----------------+--------------------------------+
|outlet_name     |outlet_tran_list                |
+----------------+--------------------------------+
|ABC Bank outlet1|[Deposit, Deposit, ACH transfer]|
|PQR Bank outlet1|[Transfer, Check deposit]       |
|STU Bank outlet1|[Bill payment, Money order]     |
|UVW Bank outlet1|[ATM withdrawal, Payroll]       |
|XYZ Bank outlet1|[Withdrawal, Online purchase]   |
+----------------+--------------------------------+

root
 |-- outlet_name: string (nullable = true)
 |-- outlet_tran_list: array (nullable = false)
 |    |-- element: string (containsNull = false)
</pre>
</td>

  <td>
  <pre>
+----------------+----------------------------+
|outlet_name     |outlet_tran_list_as_str_type|
+----------------+----------------------------+
|ABC Bank outlet1|Deposit ACH transfer        |
|PQR Bank outlet1|Transfer Check deposit      |
|STU Bank outlet1|Bill payment Money order    |
|UVW Bank outlet1|ATM withdrawal Payroll      |
|XYZ Bank outlet1|Withdrawal Online purchase  |
+----------------+----------------------------+

root
 |-- outlet_name: string (nullable = true)
 |-- outlet_tran_list_str_type: string (nullable = false)
</pre></td>

</tr></table>

  **give me the string instead of array type without comma in the value and without duplicates?**


# The point

In [42]:
from pyspark.sql import functions as f
#step: 1 remove duplicates inside array (col: outlet_tran_list)
#step: 2 array to string type conversion with delimitor change
arr_to_str_df=outlet_df.withColumn("outlet_tran_list_as_str_type"\
                                  ,f.concat_ws(" ",f.array_distinct("outlet_tran_list")))

#finally print the result for confirmation
arr_to_str_final_df=arr_to_str_df.select("outlet_name","outlet_tran_list_as_str_type")
arr_to_str_final_df.show(10,False)
arr_to_str_final_df.printSchema()

+----------------+----------------------------+
|outlet_name     |outlet_tran_list_as_str_type|
+----------------+----------------------------+
|ABC Bank outlet1|Deposit ACH transfer        |
|PQR Bank outlet1|Transfer Check deposit      |
|STU Bank outlet1|Bill payment Money order    |
|UVW Bank outlet1|ATM withdrawal Payroll      |
|XYZ Bank outlet1|Withdrawal Online purchase  |
+----------------+----------------------------+

root
 |-- outlet_name: string (nullable = true)
 |-- outlet_tran_list_as_str_type: string (nullable = false)



# In detail

In [43]:

#step: 1 remove duplicates inside array (col: outlet_tran_list)
outlet_df.show(10,False)
outlet_df.printSchema()
d_arr_df=outlet_df.withColumn("outlet_tran_list_with_dedup"\
                                  ,f.array_distinct("outlet_tran_list")).select("outlet_name","outlet_tran_list_with_dedup")
 #output of step 1
d_arr_df.show(10,False)
d_arr_df.printSchema()

#step: 2 array to string type with delimitor change
str_df=d_arr_df.withColumn("outlet_tran_list_as_str_type"\
                             ,f.concat_ws(" ","outlet_tran_list_with_dedup")).select("outlet_name","outlet_tran_list_as_str_type")

#final output of step 2
str_df.show(10,False)
str_df.printSchema()


+----------------+--------------------------------+
|outlet_name     |outlet_tran_list                |
+----------------+--------------------------------+
|ABC Bank outlet1|[Deposit, Deposit, ACH transfer]|
|PQR Bank outlet1|[Transfer, Check deposit]       |
|STU Bank outlet1|[Bill payment, Money order]     |
|UVW Bank outlet1|[ATM withdrawal, Payroll]       |
|XYZ Bank outlet1|[Withdrawal, Online purchase]   |
+----------------+--------------------------------+

root
 |-- outlet_name: string (nullable = true)
 |-- outlet_tran_list: array (nullable = false)
 |    |-- element: string (containsNull = false)

+----------------+-----------------------------+
|outlet_name     |outlet_tran_list_with_dedup  |
+----------------+-----------------------------+
|ABC Bank outlet1|[Deposit, ACH transfer]      |
|PQR Bank outlet1|[Transfer, Check deposit]    |
|STU Bank outlet1|[Bill payment, Money order]  |
|UVW Bank outlet1|[ATM withdrawal, Payroll]    |
|XYZ Bank outlet1|[Withdrawal, Online purc

# The explanation

<table style="background:red; width:85%; border:1px solid black;">
<tr style="color:red">
  <td>1.what have u(input)</td>
  <td>cust_df</td>
</tr>
<tr>
  <td colspan="3"><code>cust_df.<..></code></td>
</tr>
<tr>
  <td>2.what do u want</td>
  <td>new column as outlet_tran_list</td>
  <td>then use withColumn</td>
</tr>
<tr>
<td colspan="3">cust_df<code>.withColumn("outlet_tran_list",<..>)</code></code></td>
</tr>
<tr>
  <td>3.What for?</td>
  <td>to store comma separted tran_type values list</td>
  <td>then use collect_list</td>
</tr>
<tr>
  <td>4.from which column</td>
  <td>tran_type</td>
  <td></td>
</tr>
<tr>
<td colspan="3">cust_df.withColumn("outlet_tran_list",<code> f.collect_list("tran_type").<..>)</code></td>
</tr>
<tr>
  <td>5.how u want it</td>
  <td>for each outlet_name unique value</td>
  <td>then use over(Window.partitionBy("outlet_name")).<..></td>
</tr>
<tr><td colspan="3">cust_df.withColumn("outlet_tran_list", f.collect_list("tran_type")<code>.over(Window.partitionBy("outlet_name"))</code>)</td>
<tr>
  <td> 6.where do u want store result</td>
  <td>outlet_df</td>
</tr>
<tr>
  <td colspan="3" style="background:red;"><code>outlet_df=</code>cust_df.withColumn("outlet_tran_list", f.collect_list("tran_type").over(Window.partitionBy("outlet_name")))
  </td>
</tr>
</table>

The End

---



---



---

